In [1]:
import numpy as np


from scipy.optimize import linprog
from scipy.optimize import minimize_scalar
from scipy.optimize import Bounds
from scipy.optimize import LinearConstraint

# Матричные игры

## Задание матрицы игры

In [2]:
X = [[2, 4, 4, 2],
    [1, 2, 3, 10],
    [5, 5, 4, 7],
    [3, 8, 7, 6]]

### Условия для первого игрока

In [3]:
X1 = np.transpose(X)
for i in X1:
    r = ''
    for j in range(0, len(i)):
        r += str(i[j]) + '*X' + str(j+1)
        if j != (len(i)-1):
            r+='+'
    print(r + '⩾1')

2*X1+1*X2+5*X3+3*X4⩾1
4*X1+2*X2+5*X3+8*X4⩾1
4*X1+3*X2+4*X3+7*X4⩾1
2*X1+10*X2+7*X3+6*X4⩾1


### Условия для второго игрока

In [4]:
for i in X:
    r = ''
    for j in range(0, len(i)):
        r += str(i[j]) + '*X' + str(j+1)
        if j != (len(i)-1):
            r+='+'
    print(r + '⩽1')

2*X1+4*X2+4*X3+2*X4⩽1
1*X1+2*X2+3*X3+10*X4⩽1
5*X1+5*X2+4*X3+7*X4⩽1
3*X1+8*X2+7*X3+6*X4⩽1


## Реализция алгоритма решения матричной игры путём сведения к ЗЛП

In [3]:
def matr_game_sol_lin(X):
    Z = np.array(X)
    if np.any(np.array(X) < 0):
        Z += abs(Z.min())
    
    Z_A = np.transpose(-1*Z)
    res_A = linprog([1]*Z_A.shape[0], A_ub = Z_A, b_ub = [-1]*Z_A.shape[0], method = 'Simplex')
    
    res_B = linprog([-1]*Z.shape[0], A_ub = Z, b_ub = [1]*Z.shape[0], method = 'Simplex')
    
    return [res_A, res_B]

## Запуск алгоритма и подсчёт цены игры

In [4]:
A_opt, B_opt = matr_game_sol_lin(X)
v = 1/A_opt.fun
print('Цена игры v равна: ' + str(v))

Цена игры v равна: 4.6


## Вычисление смешанных стратегий игроков

In [5]:
p = np.array(A_opt.x)*v
q = np.array(B_opt.x)*v

In [6]:
print('Стратегия для игрока 1:')
for i in range(0, len(p)):
    print("   p" + str(i+1) + " = " + str(round(p[i], 2)))

Стратегия для игрока 1:
   p1 = 0.0
   p2 = 0.0
   p3 = 0.8
   p4 = 0.2


In [7]:
print('Стратегия для игрока 2:')
for i in range(0, len(p)):
    print("   q" + str(i+1) + " = " + str(round(q[i], 2)))

Стратегия для игрока 2:
   q1 = 0.6
   q2 = 0.0
   q3 = 0.4
   q4 = 0.0


# Осторожные стратегии

In [31]:
n = 1000

In [32]:
def U(x1, x2):
    return 2*x1*x2 + (1 - 2*x1)*(1 - 3*x2)

In [33]:
def benchmark_1(x2):
    def decorator(func):
        def wrapped(x1):
            return U(x1, x2)
        return wrapped
    return decorator

In [34]:
def benchmark_2(x1):
    def decorator(func):
        def wrapped(x2):
            return U(x1, x2)
        return wrapped
    return decorator

In [35]:
X1 = np.arange(0, 1 + 1/n, 1/n)
X2 = np.arange(0, 1 + 1/n, 1/n)
X1 = np.delete(X1, np.where(X2 == 0.375)[0]) 
X2 = np.delete(X2, np.where(X1 == 0.25)[0]) 

В некоторых точках функция выигрыша переходит в константу. Данные значения мы уберем из нашего анализа. (Для игрока 1 0.3 и для игрока 2 0.2)

## Без ограничений

Для задачи без ограничений, с учётом вида функции и ограничений на переменные, мы не получим рационального интерпретируемого результата. Так, функция при фиксации одной из переменных будет иметь линейный вид, максимума или минимума для задачи оптимизации существовать не будет, супренум или инфинум будет равен +-∞. В таком случае первому игроку будет смысл выбирать число как можно больше, а второму как можно меньше. Существует вариант когда в зависимости от значения второй переменной функция перобразовуеться в константу. Тогда другому игроку безразличен выбор стратегии, он всегда получит фиксированный результат.

## С ограничениями

### Для первого игрока

In [36]:
X1_max = []
F_max = []
for i in X2:
    @benchmark_1(x2 = i)
    def U1(x1):
        pass
    res = minimize_scalar(lambda x1: -U1(x1), bounds = (min(X1), max(X1)), method='bounded')
    X1_max.append(res.x)
    F_max.append(-res.fun)

In [37]:
X1_opt = []
U_minmax = min(F_max)
for i in range(0, len(F_max)):
    if F_max[i] <= U_minmax:
        if X1_max[i] not in X1_opt:
            X1_opt.append(X1_max[i])

if len(X1_opt) == 1:
    print('Аккуратная стратегия игрока 1 : {0}'.format(round(X1_opt[0], 2)))
else:
    print('Аккуратная стратегия игрока 1 расположена на промежутке : {0}-{1}'.format(
        round(min(X1_opt), 2), round(max(X1_opt), 2)))

Аккуратная стратегия игрока 1 : 0.0


In [38]:
for i in range(0, len(X2)):
    print(str(round(X2[i],5))+'   ' + str(round(X1_max[i], 5))+'   '+str(round(F_max[i], 5)))

0.0   1e-05   0.99999
0.001   1e-05   0.99699
0.002   1e-05   0.99399
0.003   1e-05   0.99099
0.004   1e-05   0.98799
0.005   1e-05   0.98499
0.006   1e-05   0.98199
0.007   1e-05   0.97899
0.008   1e-05   0.97599
0.009   1e-05   0.97299
0.01   1e-05   0.96999
0.011   1e-05   0.96699
0.012   1e-05   0.96399
0.013   1e-05   0.96099
0.014   1e-05   0.95799
0.015   1e-05   0.95499
0.016   1e-05   0.95199
0.017   1e-05   0.94899
0.018   1e-05   0.94599
0.019   1e-05   0.94299
0.02   1e-05   0.93999
0.021   1e-05   0.93699
0.022   1e-05   0.93399
0.023   1e-05   0.93099
0.024   1e-05   0.92799
0.025   1e-05   0.92499
0.026   1e-05   0.92199
0.027   1e-05   0.91899
0.028   1e-05   0.91599
0.029   1e-05   0.91299
0.03   1e-05   0.90999
0.031   1e-05   0.90699
0.032   1e-05   0.90399
0.033   1e-05   0.90099
0.034   1e-05   0.89799
0.035   1e-05   0.89499
0.036   1e-05   0.89199
0.037   1e-05   0.88899
0.038   1e-05   0.88599
0.039   1e-05   0.88299
0.04   1e-05   0.87999
0.041   1e-05   0.8769

В меру технических особенностей мы получаем такой приближенный результат. На самом деле, для стратегии игрока 0.2 не играет роли стратегия игрока 1, выиграш всегда будет равен 0.4.

In [25]:
print(U(0.9, 0.2))
print(U(0.3, 0.2))

0.04000000000000009
0.27999999999999997


In [39]:
plot = []
for i in X2:
    plot.append(U(X1_opt[0], i))

In [40]:
from plotly.offline import iplot, init_notebook_mode
from plotly import graph_objs as go
init_notebook_mode(connected = True)

trace = go.Scatter(x = X2, y = plot, name = 'F glob'
                   , line=dict(color="#000080", width=4), marker=dict(color="#FF00FF", size=10))
layout = dict(title = '<b>График изменения выигрыша игрока 1 от стратегии игрока 2 при его оптимальной стратегии</b>')
fig = dict(data = trace, layout = layout)
iplot(fig, show_link=True)

### Для второго игрока

In [41]:
X2_min = []
F_min = []
for i in X1:
    @benchmark_2(x1 = i)
    def U2(x2):
        pass
    res = minimize_scalar(U2, bounds = (min(X2), max(X2)), method='bounded')
    X2_min.append(res.x)
    F_min.append(res.fun)

In [48]:
X2_opt = []
U_minmax = max(F_min)
for i in range(0, len(F_min)):
    if abs(F_min[i] - U_minmax) < 0.001:
        if X2_min[i] not in X2_opt:
            X2_opt.append(X2_min[i])

if len(X2_opt) == 1:
    print('Аккуратная стратегия игрока 2 : {0}'.format(round(X2_opt[0], 2)))
else:
    print('Аккуратная стратегия игрока 2 расположена на промежутке : {0}-{1}'.format(
        round(min(X2_opt), 2), round(max(X2_opt), 2)))

Аккуратная стратегия игрока 2 : 0.0


In [43]:
for i in range(0, len(X2)):
    print(str(round(X1[i],5))+'   ' + str(round(X2_min[i], 5))+'   '+str(round(F_min[i], 5)))

0.0   0.99999   -1.99998
0.001   0.99999   -1.99398
0.002   0.99999   -1.98798
0.003   0.99999   -1.98198
0.004   0.99999   -1.97598
0.005   0.99999   -1.96998
0.006   0.99999   -1.96398
0.007   0.99999   -1.95798
0.008   0.99999   -1.95198
0.009   0.99999   -1.94598
0.01   0.99999   -1.93998
0.011   0.99999   -1.93398
0.012   0.99999   -1.92798
0.013   0.99999   -1.92198
0.014   0.99999   -1.91598
0.015   0.99999   -1.90998
0.016   0.99999   -1.90398
0.017   0.99999   -1.89798
0.018   0.99999   -1.89198
0.019   0.99999   -1.88598
0.02   0.99999   -1.87998
0.021   0.99999   -1.87398
0.022   0.99999   -1.86798
0.023   0.99999   -1.86198
0.024   0.99999   -1.85598
0.025   0.99999   -1.84998
0.026   0.99999   -1.84398
0.027   0.99999   -1.83798
0.028   0.99999   -1.83198
0.029   0.99999   -1.82598
0.03   0.99999   -1.81998
0.031   0.99999   -1.81398
0.032   0.99999   -1.80798
0.033   0.99999   -1.80198
0.034   0.99999   -1.79598
0.035   0.99999   -1.78998
0.036   0.99999   -1.78398
0.037 

Для игрока 1 и его стратегии 0.3 не играет роли стратегия игрока 2, выиграш всегда будет равен 0.4.

In [44]:
print(U(0.3, 0.9))
print(U(0.3, 0.2))

-0.14000000000000012
0.27999999999999997


In [46]:
plot = []
for i in X1:
    plot.append(U(X2_opt[0], i))

In [47]:
from plotly.offline import iplot, init_notebook_mode
from plotly import graph_objs as go
init_notebook_mode(connected = True)

trace = go.Scatter(x = X2, y = plot, name = 'F glob'
                   , line=dict(color="#000080", width=4), marker=dict(color="#FF00FF", size=10))
layout = dict(title = '<b>График изменения выигрыша игрока 2 от стратегии игрока 1 при его оптимальной стратегии</b>')
fig = dict(data = trace, layout = layout)
iplot(fig, show_link=True)